In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import collections
%matplotlib inline

In [2]:

DATA_PATH = './data/medical/'
BINDINGDB = 'BindingDB_All.tsv'
ChEMBL = 'BindingDB_ChEMBL.tsv'

def load_BindingDB(file):
    file_path = DATA_PATH + file

    ## load the first row to get the number of columns
    df_first_row = pd.read_csv(file_path, sep='\t', nrows=1)

    ## number of columns to use
    df = pd.read_csv(file_path, sep='\t', header=0, usecols=range(df_first_row.shape[1]))
    return df

df = load_BindingDB(BINDINGDB)

/tmp/ipykernel_4657/4248568440.py:12: DtypeWarning: Columns (8,9,10,11,12,13,15,17,20,21,22,23,27,28,32,33,35,36,45,46,47,48,50,51,52,53,54,55,57,58,59,60,62,63,64,65,66,67,69,70,71,72,74,75,76,77,78,79,81,82,83,86,87,88,89,90,91,93,94,95,98,99,100,101,102,103,105,106,107,110,111,112,113,114,115,117,118,119,122,123,124,125,126,127,129,130,131,134,135,136,137,138,139,141,142,143,146,147,148,149,150,151,153,154,155,158,159,160,161,162,163,165,166,167,170,171,172,173,174,175,177,178,179,182,183,184,185,186,187,189,190,191) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, sep='\t', header=0, usecols=range(df_first_row.shape[1]))


In [ ]:
print('Number of unique dois:', len(unique_dois))
if os.path.exists(DOI_DF_PATH):
    df_dois = pd.read_pickle(DOI_DF_PATH)
    print('Loaded doi_df from', DOI_DF_PATH)
else:
    print(f'No DOI dataframe found at {DOI_DF_PATH}. Creating a new one...')
    cols = ['fetched', 'title', 'abstract', 'year', 'month', 'day']
    unique_dois = df['Article DOI'].dropna().unique()

    # Create an empty dataframe with doi as index and cols with nans
    df_dois = pd.DataFrame(index=unique_dois, columns=cols)
    df_dois['fetched'] = False

    df_dois.index.name = 'doi'
    df_dois.head()

Number of unique dois: 40737


,fetched,title,abstract,year,month,day
doi,,,,,,
10.1021/jm9602571,False,NaN,NaN,NaN,NaN,NaN
10.1016/j.chembiol.2013.07.014,False,NaN,NaN,NaN,NaN,NaN
10.1021/jm9507183,False,NaN,NaN,NaN,NaN,NaN
10.1126/science.2200122,False,NaN,NaN,NaN,NaN,NaN
10.1021/ja00082a004,False,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Use api to get info about article based on doi
import requests
from tqdm import tqdm

def fetch_article_metadata(doi):
    url = f"https://api.crossref.org/works/{doi}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["message"]
        
        title_list = data.get("title", [])
        title = title_list[0] if len(title_list) > 0 else pd.NA
        abstract = data.get("abstract", pd.NA)
        # Get published year, month and day
        published_date = data.get("published-print", None)
        if published_date:
            published_date = published_date["date-parts"][0]
        else:
            published_date = []
        year = published_date[0] if len(published_date) > 0 else pd.NA
        month = published_date[1] if len(published_date) > 1 else pd.NA
        day = published_date[2] if len(published_date) > 2 else pd.NA

        
        
        return {
            "title": title,
            "abstract": abstract,
            "year": year,
            "month": month,
            "day": day
        }
    else:
        print(f"Failed to fetch data for DOI {doi}. Status code: {response.status_code}")
        return None

# Example usage
doi = np.random.choice(unique_dois)
metadata = fetch_article_metadata(doi)

if metadata:
    print("Title:", metadata["title"])
    print("Abstract:", metadata["abstract"])


Title: <i>N</i>1-Azinylsulfonyl-1<i>H</i>-indoles: 5-HT6 Receptor Antagonists with Procognitive and Antidepressant-Like Properties
Abstract: No abstract available


In [45]:
import time
import random
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm

def fetch_and_update(doi):
    if df_dois.loc[doi, 'fetched']:
        return None

    # Fetch article metadata with a random delay
    article_info = fetch_article_metadata(doi)
    if article_info is None:
        df_dois.loc[doi, 'fetched'] = True
        return None
    for key, value in article_info.items():
        df_dois.loc[doi, key] = value
    df_dois.loc[doi, 'fetched'] = True
    
    # Sleep for a short, random time to avoid triggering rate limits
    time.sleep(random.uniform(0.2, 1))

with ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(fetch_and_update, df_dois.index), total=len(df_dois.index)))


  5%|▍         | 1977/40737 [00:00<00:15, 2450.82it/s]

Failed to fetch data for DOI 28357. Status code: 404
Failed to fetch data for DOI 10.1002/1098-2396(20001215)38:4. Status code: 404
Failed to fetch data for DOI 10.1002/1098-2396(20010101)39:1. Status code: 404
Failed to fetch data for DOI 10.1002/1520-636X(2001)13:3. Status code: 404


  6%|▌         | 2385/40737 [01:16<1:52:32,  5.68it/s]

Failed to fetch data for DOI 0.1007/s00044-005-0126-y. Status code: 404


 72%|███████▏  | 29378/40737 [1:27:40<33:42,  5.62it/s]  

Failed to fetch data for DOI 10.1097/01.aids.0000060380.78202.b5. Status code: 404


100%|██████████| 40737/40737 [2:03:47<00:00,  5.48it/s]  


In [ ]:
# replace No abstract available	 wiht pd.NA
df_dois.head()

,fetched,title,abstract,year,month,day
doi,,,,,,
10.1021/jm9602571,True,"Cyclic HIV Protease Inhibitors: Synthesis, Co...",<NA>,1996,1,1
10.1016/j.chembiol.2013.07.014,True,Substrate Envelope-Designed Potent HIV-1 Prote...,<NA>,2013,9,<NA>
10.1021/jm9507183,True,"A Novel, Picomolar Inhibitor of Human Immunode...",<NA>,1996,1,1
10.1126/science.2200122,True,"Design, Activity, and 2.8 Å Crystal Structure ...",<jats:p>\n A two-fold (\n ...,1990,8,3
10.1021/ja00082a004,True,Influence of stereochemistry on activity and b...,<NA>,1994,2,<NA>


In [53]:
# Show percentage of missing values per column
df_dois.isnull().mean()



fetched     0.000000
title       0.000687
abstract    0.981786
year        0.002848
month       0.004198
day         0.627292
dtype: float64

In [54]:
# Summary of all df_dois columns
df_dois.describe(include='all')

,fetched,title,abstract,year,month,day
count,40737,40709,742,40621,40566,15183
unique,1,40699,742,50,12,31
top,True,Factor XI Activation Inhibitors for Treating T...,<p>After a century since the first antimonial-...,2013,1,1
freq,40737,2,1,2104,4173,5480


In [1]:
if df_dois.fetched.all():
    print('All articles were fetched. Saving df_dois')
    df_dois.to_pickle()
else:
    print('Not all articles were fetched. Saving work in progress')
    df_dois.to_pickle(DATA_PATH + 'df_dois_wip.pkl')

NameError: name 'df_dois' is not defined